# INM706 Deep Learning for Sequence Analysis
### Sarah Rhalem (190051884) & Stelios Kliafas (########)

Draft Notes/ Working comments:
Dataset >> Problem +Evaluation Metric >> Model

In [1]:
# Import libraries



#### Dataset - Import & pre-processing

#### Model

#### Training & Validation

#### Hyperparameter Tuning

#### Evaluation